This is a script for geocoding and plotting spatial information from an EXCEL spreadsheet with an "Addresses" column. The API used is **GeoNames.**

GeoNames is mainly using REST APIs. It offers 40 different webservices.

**Geocoder** for Python supports the following ones:

*   (geocoding) retrieve GeoNames’s geocoded data from a query string, and 
various filters
*   (details) retrieve all geonames data for a given geonames_id
*   (children) retrieve the hierarchy of a given geonames_id
*   (hierarchy) retrieve all children for a given geonames_id

Full documentation: https://geocoder.readthedocs.io/providers/GeoNames.html

The first step is to get COLAB working:


In [ ]:
## mount drive
from google.colab import drive
drive.mount("/content/drive")

A file path needs to be defined for storing input or output files linked with this script:

In [ ]:
directory="/content/drive/My Drive/Colab_FASoS/" ## add your own folder name

Now we can install packages that are not part of Python's standard distribution but are necessary for geocoding and plotting maps. There will most likely be a dependency error for NumPy, but the script should still work.

In [ ]:
## install packages that are not part of Python's standard distribution

!pip install geocoder
!pip install basemap
!pip install ipyleaflet
!pip install geojson

Now that all packages are installed, we can read the input data (in this case from Github or Google Drive) and display the content in a table.

In [ ]:
## import relevant packages for geocoding as well as reading and writing data
import pandas as pd
import numpy as np # e.g. for working with empty cells
import geocoder
# command needed for correct plotting in Jupyter Notebooks:
%matplotlib inline 
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import os
import json
from geojson import Feature, FeatureCollection, Point

## geocode data from spreadsheet

## input addresses in EXCEL format and read
## OPTION 1: from Github as raw file
##infile="https://github.com/MonikaBarget/GeoHumTutorials/blob/master/Colab_Geocoding/Addresses_AP3.xlsx?raw=true"

## OPTION 2: from your Google Drive as EXCEL FILE
infile=directory+"new-places.xlsx"

## OPTION 3: from your Google Drive as CSV FILE
#infile=directory+"ATLASSES & COSMOGRAPHIES_utf-8.csv"

## read if EXCEL
addresses_df = pd.read_excel(infile)
display(addresses_df)

## read if CSV
#addresses_df = pd.read_csv(infile, encoding="utf-8", delimiter=";") # delimiter=None # encoding=None # encoding_errors='strict'
#display(addresses_df)

Now we will use the Pandas package to read the content of the address column to a so-called DataFrame. A DataFrame is a data structure that organizes data into a 2-dimensional table of rows and columns, much like a spreadsheet. This 2-dimensional structure is often used to manipulate data with programming languages. Our "manipulation" is the act of geocoding.

In [ ]:
# read information from address column to dataframe
addresses_df["addresses_full"] = addresses_df["Addresses"].astype(str) + ", " + addresses_df["continent"].astype(str)
addresses = addresses_df["addresses_full"].values.tolist() # add name of column to geocode

latitudes = []
longitudes = []
ids = []
g_addresses = []

# geocode each address in file if no ID found at all

if "ids" not in addresses_df: 
	for address in addresses:
		try:
			g = geocoder.geonames(address, key="Mob2023", featureClass='P') # http://www.geonames.org/source-code/javadoc/org/geonames/FeatureClass.html

			if g and len(g):
				geonames_address = g.address
				longitude = g.lng
				latitude = g.lat
				id = g.geonames_id
				#print(geonames_address, longitude, latitude, id) # OPTIONAL: print individual output
			else:
				geonames_address = "NaN"
				longitude = "NaN"
				latitude = "NaN"
				id = "NaN"

			# add information to lists
			g_addresses.append(geonames_address)
			latitudes.append(latitude)
			longitudes.append(longitude)
			ids.append(id)

		except ValueError:
			print("No more data in file.") 
	 
	# write information to new columns in dataframe
	addresses_df["latitudes"] = latitudes
	addresses_df["longitudes"] = longitudes
	addresses_df["ids"] = ids
	addresses_df["geonames address"] = g_addresses
	 
# geocode only empty values if ID column is found

else:
	main_df=addresses_df[addresses_df["ids"].notna()]
	sub_df=addresses_df[addresses_df['ids'].isna()] # fill dataframe where ID cell has no value
	display(sub_df)
	sub_addresses = sub_df["Addresses"].values.tolist()
	for sub_address in sub_addresses:
		try:
			g = geocoder.geonames(sub_address, key="Mob2023", featureClass='P') # http://www.geonames.org/source-code/javadoc/org/geonames/FeatureClass.html

			if g and len(g):
				geonames_address = g.address
				longitude = g.lng
				latitude = g.lat
				id = g.geonames_id
				#print(geonames_address, longitude, latitude, id) # OPTIONAL: print individual output
			else:
				geonames_address = "NaN"
				longitude = "NaN"
				latitude = "NaN"
				id = "NaN"

			# add information to lists
			g_addresses.append(geonames_address)
			latitudes.append(latitude)
			longitudes.append(longitude)
			ids.append(id)

		except ValueError:
			print("No more data in file.")

	# write information to new columns in sub-dataframe
	sub_df["latitudes"] = latitudes
	sub_df["longitudes"] = longitudes
	sub_df["ids"] = ids
	sub_df["geonames address"] = g_addresses

	addresses_df = pd.concat([main_df, sub_df])

print("All addresses geocoded!")


If all addresses have been successfully geocoded, the next step is to check the geocoding and write the results to a new EXCEL file. 


In [ ]:
# view geocoded data
display(addresses_df)

# write geocoded places to new file
addresses_df.to_excel(directory+"Mainz2.xlsx")

addresses_df.to_csv(directory+"Mainz2.csv")

Our geocoded data have been written to a new EXCEL file, which is handy for further (manual) data cleaning and data enrichment. But EXCEL is unfortunately not a file format which GIS applications can handle. This is why we also need to export our geocoded data to GeoJSON.

The conversion of a DataFrame to GeoJSON follows the instructions in the following tutorial by Geoff Boeing:

https://notebook.community/captainsafia/nteract/applications/desktop/example-notebooks/pandas-to-geojson

In [ ]:
# convert coordinates to floats

addresses_df['latitudes'] = addresses_df['latitudes'].astype(float)
addresses_df['longitudes'] = addresses_df['longitudes'].astype(float)

# ignore places that have not been geocoded

df_geo = addresses_df.dropna(subset=['latitudes', 'longitudes'], axis=0, inplace=False)

# combine information in GeoJSON fromat

def df_to_geojson(df, properties, lat='latitudes', lon='longitudes'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

cols = ['Addresses', 'ids', 'geonames address'] # make sure that your column with place names is selected
geojson = df_to_geojson(df_geo, cols)

with open(directory+'Mainz2.geojson', 'w', encoding='utf-8') as f:
    json.dump(geojson, f, ensure_ascii=False)

Your Google Drive should now contain a file with the "geojson" file ending. We can check if this file has been created and if it is well-formed.

In [ ]:
## double-check if GeoJSON file has been created and is well-formed

# load GeoJSON data

with open(directory+'Mainz2.geojson', 'r') as f2:
    data = json.load(f2)
    print(data)

Now we can plot the geocoded data to an interactive map. The code below is partly based on an Ipyleaflet Tutorial provided by the *Carpentries Incubator*:

https://carpentries-incubator.github.io/jupyter_maps/01-introduction/index.html

In [ ]:
# plot map from geocoded data and add labels for all places on map

from ipyleaflet import Map, Marker, Popup
from ipywidgets import HTML

# Create a map centered on a specific location
map = Map(center=(55, 7), zoom=3)

# print(data) # optional to check if GeoJSON file is read correctly

# Loop over the features and create a marker for each one
for feature in data['features']:
    # Get the coordinates and place name from the feature properties
    coords = feature['geometry']['coordinates']
    place_name = feature['properties']['Addresses'] # double-check name of "place name" column
    #print(place_name) # optional to check if place names are correctly displayed

    # Create a marker

    marker = Marker(location=(coords[1], coords[0]))

    # Create pop-up for each location
        
    popup = Popup(
        location=(coords[1], coords[0]),
        close_button=False,
        auto_close=False,
        close_on_escape_key=False,
    )
    popup.children = str(place_name)[:20] # truncate place names to first "n" characters
    print(popup.children)
    message= HTML()
    message.value = popup.children
    marker.popup = message

    # Add the marker to the map
    map.add_layer(marker)

# Display the map
map



Congratulations, you have just plotted a new map! At the moment, the map only has markers for the point geometries but no pop-up labels. To embed those, other Python packages will need to be imported first. I will add pop-ups in the next development step. 

Notebook created by: Monika Barget

Latest update: 26 January 2023